In [291]:
import pandas as pd
import re
import math
import numpy as np
import matplotlib.pyplot as plt
import copy
from random import shuffle,randint

In [484]:
file = open('24x40.txt','r')
N, M = list(map(int,file.readline().split()))
data = []
N1 = 0
for i in range(N):
    string = [0] * M
    vehicle = list(map(int,file.readline().split()))
    for s in vehicle[1:]:
        string[int(s)-1] = 1
        N1+=1
    data.append(string)

In [480]:
def print_matrix(mat):
    for d in mat:
        for c in d:
            if c==0:
                print("_", end = ' ')
            else:
                print(c,  end = ' ')
        print('\n', end='')
        #print(*d, end = '\n')

In [481]:
print_matrix(data)

_ _ _ _ _ _ _ _ 1 _ _ _ _ _ _ _ 1 _ 1 _ _ _ _ _ _ _ _ _ _ _ 1 _ 1 _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ 1 1 _ _ _ _ _ _ _ 1 _ _ _ _ _ _ _ _ _ _ 1 _ _ _ _ _ _ 1 
_ _ _ _ _ _ _ _ _ _ _ 1 _ _ 1 _ _ _ _ _ _ _ 1 1 _ _ _ _ _ _ _ _ _ 1 _ _ _ _ _ _ 
_ _ _ _ _ _ _ 1 _ _ 1 _ _ _ _ _ _ _ _ _ 1 _ _ _ _ _ _ 1 _ _ _ _ _ _ _ _ _ 1 _ _ 
_ _ _ _ 1 1 _ _ _ 1 _ 1 _ _ _ _ _ _ _ _ _ 1 _ _ _ _ _ _ _ _ _ _ _ _ 1 _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ 1 _ _ _ _ _ _ _ 1 _ _ _ _ _ _ _ 1 _ _ _ 1 _ _ _ _ _ _ _ 1 _ _ 
_ 1 _ _ _ _ _ _ _ _ _ _ 1 _ _ _ _ _ _ _ _ _ _ _ 1 _ _ 1 _ _ _ 1 _ _ _ _ _ _ _ _ 
_ _ _ 1 1 _ _ _ _ _ _ _ _ _ _ 1 _ _ _ _ _ _ _ _ _ 1 _ _ _ 1 _ _ _ 1 _ _ _ 1 1 _ 
_ _ _ _ _ 1 _ _ _ _ _ 1 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 1 _ _ _ _ _ 1 1 
_ _ _ _ _ 1 _ _ 1 _ _ _ _ _ _ _ _ _ _ 1 _ _ _ 1 _ 1 _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ 1 _ _ _ _ _ _ 1 _ _ _ _ 1 _ _ _ _ _ _ 1 1 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ 1 _ _ _ _ _ _ _ _ _ _ _ _ _ 1 _ 1 _ 1 _ 1 _ _ 1 _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ 1 _ _ _ _ _ _ _ _ 1 _ _ 

In [38]:
class Cluster(object):
    def __init__(self, vehicles=[], details=[]):
        self._vehicles: list = vehicles
        self._details: list = details
        result = self.get_matrix()
        self.N1_in: int = result[1]
        self.N0_in: int = result[2]
        self.matrix = result[0]
    def __repr__(self):
        s=""
        for m in self.matrix:
            s+= str(m).replace(","," ")[1:-1] +'\n'
        return f"\nCustomer {self._vehicles, self._details, self.N1_in, self.N0_in, self.object_function()} \n{s[:-1]}"
    
    def get_matrix(self):
        mat = []
        for v in self._vehicles:
            line = []
            for d in self._details:
                line.append(data[v][d])
            mat.append(line)
            N1_in = sum([sum(m) for m in mat])
            N0_in = len(self._vehicles)*len(self._details) - N1_in
        return mat, N1_in, N0_in
    
    def object_function(self):
        return self.N1_in / (N1 + self.N0_in)
    
    def delete_vehicle(self,number):
        i = self._vehicles.index(number)
        n1 = sum(self.matrix[i][:])
        n0 = len(self._details) - n1
        self.N1_in -= n1
        self.N0_in -= n0
        self.matrix = self.matrix[:i] + self.matrix[i+1:] 
        self._vehicles.pop(i)
    
    def add_vehicle(self,number):
        line = []
        for d in self._details:
            line.append(data[number][d])
        n1 = sum(line)
        n0 = len(self._details) - n1
        self.N1_in += n1
        self.N0_in += n0
        self.matrix.append(line)
        self._vehicles.append(number)
        
    def delete_detail(self,number):
        i = self._details.index(number)
        n1 = sum(r[i] for r in self.matrix)
        n0 = len(self._vehicles) - n1
        self.N1_in -= n1
        self.N0_in -= n0
        [r.pop(i) for r in self.matrix]
        self._details.pop(i)
    
    def add_detail(self,number):
        line = []
        for v in self._vehicles:
            line.append(data[v][number])
        n1 = sum(line)
        n0 = len(self._vehicles) - n1
        self.N1_in += n1
        self.N0_in += n0
        [r.append(line.pop(0)) for r in self.matrix]
        self._details.append(number)

In [361]:
def initial_solution(): 
    best_sum = 0
    K = min(N,M) // 2 
    for _ in range(20):
        shuffled_vehicle = list(range(N))
        shuffled_details = list(range(M))
        shuffle(shuffled_vehicle)
        shuffle(shuffled_details)
        solution = []
        for _ in range(K):
            veh, det, i = [], [], 0
            while (i< N//K) and (len(shuffled_vehicle)>1):
                veh.append(shuffled_vehicle.pop(0))
                i+=1
            i=0
            while (i< M//K) and (len(shuffled_details)>1):
                det.append(shuffled_details.pop(0))
                i+=1
            if (len(veh)>0) and (len(det)>0):
                solution.append(Cluster(veh, det))
        solution.append(Cluster(shuffled_vehicle, shuffled_details))
        cur_sum = sum([s.object_function() for s in solution])
        if (cur_sum > best_sum):
            best_solution = solution
            best_sum = cur_sum
    return best_solution

In [362]:
def is_feasible(solution): #проверка на то, что решение существует
    res = get_output(solution)
    veh = set(res[0])
    det = set(res[1])
    if (len(veh - det)==0) and (len(det - veh)==0):
        return True
    else:
        return veh, det

In [264]:
def get_solution_matrix(solution): #функция дальше не используется, но она помогает визуализировать распределение кластеров
    solution_matrix = [['_']*N for _ in range(M)]
    for i in range(len(solution)):
        for v in solution[i]._vehicles:
            for d in solution[i]._details:
                solution_matrix[v][d] = i + 1
    print_matrix(solution_matrix)
get_solution_matrix(best_sol)

_ _ _ _ _ 4 _ _ 4 4 4 _ _ _ _ 4 _ _ _ _ 
1 1 1 _ 1 _ 1 1 _ _ _ _ _ _ _ _ _ _ _ 1 
_ _ _ 5 _ _ _ _ _ _ _ _ 5 5 5 _ _ 5 _ _ 
_ _ _ _ _ 4 _ _ 4 4 4 _ _ _ _ 4 _ _ _ _ 
_ _ _ 5 _ _ _ _ _ _ _ _ 5 5 5 _ _ 5 _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 2 _ 
_ _ _ _ _ 4 _ _ 4 4 4 _ _ _ _ 4 _ _ _ _ 
_ _ _ 5 _ _ _ _ _ _ _ _ 5 5 5 _ _ 5 _ _ 
_ _ _ _ _ 4 _ _ 4 4 4 _ _ _ _ 4 _ _ _ _ 
_ _ _ _ _ 4 _ _ 4 4 4 _ _ _ _ 4 _ _ _ _ 
_ _ _ 5 _ _ _ _ _ _ _ _ 5 5 5 _ _ 5 _ _ 
_ _ _ _ _ _ _ _ _ _ _ 3 _ _ _ _ 3 _ _ _ 
_ _ _ _ _ 4 _ _ 4 4 4 _ _ _ _ 4 _ _ _ _ 
_ _ _ 5 _ _ _ _ _ _ _ _ 5 5 5 _ _ 5 _ _ 
1 1 1 _ 1 _ 1 1 _ _ _ _ _ _ _ _ _ _ _ 1 
_ _ _ _ _ 4 _ _ 4 4 4 _ _ _ _ 4 _ _ _ _ 
_ _ _ 5 _ _ _ _ _ _ _ _ 5 5 5 _ _ 5 _ _ 
1 1 1 _ 1 _ 1 1 _ _ _ _ _ _ _ _ _ _ _ 1 
_ _ _ 5 _ _ _ _ _ _ _ _ 5 5 5 _ _ 5 _ _ 
1 1 1 _ 1 _ 1 1 _ _ _ _ _ _ _ _ _ _ _ 1 


In [260]:
def get_output(solution, k=0): #преобразование решения в итоговый вариант
    vehicle = ['_'] * N
    details = ['_'] * M
    for i in range(len(solution)):
        for v in solution[i]._vehicles:
            vehicle[v] = i+k
        for d in solution[i]._details:
            details[d] = i+k
    return vehicle, details

In [262]:
get_output(solution,1)

([4, 1, 5, 4, 5, 2, 4, 5, 4, 4, 5, 3, 4, 5, 1, 4, 5, 1, 5, 1],
 [1, 1, 1, 5, 1, 4, 1, 1, 4, 4, 4, 3, 5, 5, 5, 4, 3, 5, 2, 1])

In [203]:
def exchange_vehicle(solution): 
    #функция для LS, позволяющая переместить строку в другой класс (перебирает все строки всех классов)
    best_sum = sum([s.object_function() for s in solution])
    best_sol = copy.deepcopy(solution)
    for cluster in solution:
        first_cluster = copy.deepcopy(cluster)
        if len(cluster._vehicles)>1:
            for vehicle in cluster._vehicles:
                for c in solution:
                    if (c._vehicles == cluster._vehicles) :
                        continue
                    second_cluster = copy.deepcopy(c)
                    c.add_vehicle(vehicle)
                    cluster.delete_vehicle(vehicle)
                    if (first_cluster.object_function() + second_cluster.object_function() \
                                    < cluster.object_function() + c.object_function()):
                        cur_sum = sum([s.object_function() for s in solution])
                        if (cur_sum > best_sum):
                            best_sol = copy.deepcopy(solution)
                            best_sum = copy.deepcopy(cur_sum)
                    cluster.add_vehicle(vehicle)
                    c.delete_vehicle(vehicle) 
    solution = (best_sol)
    return copy.deepcopy(solution), best_sum

In [204]:
def exchange_detail(solution):
    #функция для LS, позволяющая переместить столбец в другой класс (перебирает все строки всех классов)
    best_sum = sum([s.object_function() for s in solution])
    best_sol = copy.deepcopy(solution)
    for cluster in solution:
        first_cluster = copy.deepcopy(cluster)
        if len(cluster._details)>1:
            for detail in cluster._details:
                for c in solution:
                    if (c._vehicles == cluster._vehicles) :
                        continue
                    second_cluster = copy.deepcopy(c)
                    c.add_detail(detail)
                    cluster.delete_detail(detail)
                    if (first_cluster.object_function() + second_cluster.object_function() \
                                    < cluster.object_function() + c.object_function()):
                        cur_sum = sum([s.object_function() for s in solution])
                        if (cur_sum > best_sum):
                            best_sol = copy.deepcopy(solution)
                            best_sum = copy.deepcopy(cur_sum)
                    cluster.add_detail(detail)
                    c.delete_detail(detail) 
    solution = (best_sol)
    return copy.deepcopy(solution), best_sum

In [483]:
def local_search(solution):#local search, улучшающий решение
    best_sum = sum([s.object_function() for s in solution])
    print("INITIAL ", best_sum)
    old_sum = 0
    step = 0
    while(True):
        while best_sum - old_sum > 0.0001 :
            old_sum = copy.deepcopy(best_sum)
            step += 1
            solution, best_sum = exchange_vehicle(solution)
#             print("STEP:{} ADD_VEH_SUM :{}".format(step, best_sum))
        old_sum = copy.deepcopy(best_sum)
        solution, best_sum = exchange_detail(solution)
        if best_sum - old_sum > 0.0001 :
#             print("IMPROVING ADD_DET_SUM :{}".format(best_sum))
            continue
        else:
            print("END AT :{}".format(best_sum))
            break
    return copy.deepcopy(solution), best_sum    

In [171]:
def merge_clusters(cl1,cl2):
    return Cluster(cl1._vehicles + cl2._vehicles, cl1._details + cl2._details)

In [197]:
def split_cluster(cl):
    #предполагаем, что кластер разбивается на два равных по длине кластера
    shuffle(cl._vehicles)
    shuffle(cl._details)
    veh_len = len(cl._vehicles) // 2
    det_len = len(cl._details) // 2
    c1 = Cluster(cl._vehicles[:veh_len], cl._details[:det_len])
    c2 = Cluster(cl._vehicles[veh_len:], cl._details[det_len:])
    return c1,c2

In [290]:
def nice_view(sol,name): #функция для записи ответа в файл
    file = open(name, 'w+')
    res = get_output(sol)
    file.write(str(res[0])[1:-1].replace(', ', ' ') + "\n")
    file.write(str(res[1])[1:-1].replace(', ', ' ') + "\n")
    print(res[0])
    print(res[1])
    file.close()
    total_file = open('VNS_total.txt', 'a+')
    total_file.write(name[:-4] + ' ' + str(sum([s.object_function() for s in sol]))+ "\n")
    print(name[:-4] + ' ' + str(sum([s.object_function() for s in sol]))+ "\n")
    total_file.close()

In [486]:
#MAIN FUNCTION
#вся магия здесь
solution = initial_solution()
best_sum = sum([s.object_function() for s in solution])
best_sol = copy.deepcopy(solution)
while (True):
    solution, cur_sum = copy.deepcopy(local_search(solution))  
    if cur_sum > best_sum:
        best_sum = copy.deepcopy(cur_sum)
        best_sol = copy.deepcopy(solution)
#         print("\tSHAKING MERGE AT ", best_sum)
        #merge
        if len(solution)>1: #слить 2 кластера можно только если их как минимум 2 в решении
            i = randint(0, len(solution)-2)
            j = randint(i+1, len(solution)-1)
            cl = merge_clusters(solution.pop(i), solution.pop(j-1))
            solution.append(cl)
            continue
    else:   
    #split
        solution = copy.deepcopy(best_sol)
        i = randint(0, len(solution)-1)
        k = 0
        while (len(solution[i]._vehicles)<2 or len(solution[i]._details)<2): 
            #разбить кластер можно только если в нем больше 2 строк и 2 столбцов
            i = randint(0, len(solution)-1)
            k +=1
            if k > len(solution):
                break
        else:
            cls = split_cluster(solution.pop(i))
            solution += cls
        solution, cur_sum = copy.deepcopy(local_search(solution))
        if cur_sum > best_sum:
            best_sum = copy.deepcopy(cur_sum)
            best_sol = copy.deepcopy(solution)
#             print("\tSHAKING SPLIT IMPROVED TO ", best_sum)
            continue
        else:
            solution = copy.deepcopy(best_sol)
            break
print(best_sum)

INITIAL  0.1274862491673175
END AT :0.4195197998733555
INITIAL  0.431029342961846
END AT :0.45451187481162075
INITIAL  0.45700327653966377
END AT :0.4693335143103501
INITIAL  0.4546869404907534
END AT :0.4853257112158269
INITIAL  0.48328951212080423
END AT :0.49155226699065085
INITIAL  0.4926299332857085
END AT :0.5032710598905872
INITIAL  0.4726782017007328
END AT :0.49586778560621314
INITIAL  0.4485542424420755
END AT :0.5131707599143736
INITIAL  0.5131707599143736
END AT :0.5131707599143736
INITIAL  0.4903287164616431
END AT :0.5000964518951212
0.5131707599143736


In [487]:
BSOL = best_sol
BSUM = best_sum

In [488]:
is_feasible(BSOL)

True

In [489]:
get_output(BSOL)

([1, 6, 0, 8, 4, 3, 7, 3, 4, 5, 0, 5, 4, 8, 2, 2, 6, 5, 2, 7, 1, 3, 4, 7],
 [1,
  7,
  4,
  5,
  3,
  4,
  6,
  8,
  1,
  0,
  8,
  4,
  6,
  2,
  0,
  3,
  1,
  5,
  1,
  5,
  8,
  6,
  0,
  5,
  7,
  2,
  2,
  8,
  6,
  3,
  6,
  7,
  1,
  3,
  4,
  2,
  2,
  3,
  3,
  6])

In [478]:
nice_view(BSOL, '24x40.sol')

[1, 0, 1, 2, 0, 1, 1, 2, 1, 1, 2, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 1, 0, 1, 2, 1, 2, 0, 2, 0, 0]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
37x53 0.6289532711953234

